# Preparing Data for Neuralangelo

Author: Deeepwin  
Date: 08.07.2023 
***

### Literature

Related Literature:  

- [Nerualangelo Supplementary](https://research.nvidia.com/labs/dir/neuralangelo/supplementary.pdf) containing details information on results.
- [InstantNGP](https://github.com/NVlabs/instant-ngp/blob/master/docs/nerf_dataset_tips.md#COLMAP) instructions to prepare new custom image datasets using COLMAP.  

  The fox example dataset can be found in original [InstantNGP repository](https://github.com/NVlabs/instant-ngp/tree/master/data/nerf/fox). InstantNGP implementation expects initial camera parameters to be provided in a `transforms.json` file in a format compatible with [the original NeRF codebase](https://www.matthewtancik.com/nerf) contains the pose information for each image.  

  The script `colmap2nerf.py` can be used to convert a text colmap export `images.txt` to nerf format `transforms.json`. However, script require as well the COLMAP `cameras.txt` containing the intrinsic parameters of all cameras.

- [NeRF Original Paper](https://github.com/bmild/nerf#already-have-poses) showing how to load already existing poses.

- [COLMAP Output Format](https://colmap.github.io/format.html#images-txt) explains the `images.txt` which contains the poses and keypoints of all images.



### Datasets

Benchmarks for 3D Surface Reconstruction used by NeuralAngelo:

- [Tanks and Temples](https://www.tanksandtemples.org/download/): Input is images or video. Poses are calculated from reconstruction using COLMAP into a [.log](http://redwood-data.org/indoor/fileformat.html) file format.

### Installation

Run this command in a terminal to create new conda environment:     

`conda create --name sdfstudio python=3.8`

In [ ]:
# install pytorch with CUDA
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# requires nvcc
!conda install -y -c "nvidia/label/cuda-11.3.1" cuda-toolkit
!conda install -y -c "nvidia/label/cuda-11.3.1" cuda-nvcc

In [ ]:
# set path to CUDA installation
!conda env config vars set CUDA_HOME="/home/martin/anaconda3/envs/sdfstudio/"
!conda env config vars list

In [ ]:
# check CUDA versions, must be 11.3
!conda list cuda-nvcc

In [ ]:
# require version 9, change default version 'sudo -S update-alternatives --config gcc'
from getpass import getpass
p = getpass()
!echo {p} | sudo -S apt -y install g++-9
!echo {p} | sudo -S update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 9
!echo {p} | sudo -S update-alternatives --config g++
!echo {p} | sudo -S apt -y install gcc-9
!echo {p} | sudo -S update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 9
!echo {p} | sudo -S update-alternatives --config gcc
del p

In [ ]:
# install tinycudann (you might need to run this command in separate terminal in case of an error)
!pip install ninja git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

In [ ]:
# install nerfstudio as well
!pip install nerfstudio

In [ ]:
# install SDF studio
!git clone https://github.com/autonomousvision/sdfstudio.git
%cd sdfstudio
!pip install -e .
%cd ..

In [ ]:
# reinstall to ensure that torch and CUDA have same version, as previous installation steps might have changed it
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# nerf studio utilities
!ns-install-cli

#### Test SDF Studio

In [ ]:
!ns-download-data sdfstudio

In [ ]:
# train model from scratch on the dtu dataset scan65, default iteration is 20000
!ns-train neus-facto \
    --viewer.websocket-port 7009 \
    --pipeline.model.sdf-field.inside-outside False \
    --vis viewer \
    --experiment-name neus-facto-dtu65 \
    sdfstudio-data --data data/sdfstudio-demo-data/dtu-scan65

In [ ]:
# copy paste output directory
output_dir = 'outputs/neus-facto-dtu65/neus-facto/2023-07-28_122528'

In [ ]:
# extract mesh
!ns-extract-mesh --load-config $output_dir/config.yml --output-path exports/neus-facto-dtu65/mesh.ply

In [ ]:
# export as point cloud, adjust export directory
!ns-export pointcloud --load-config $output_dir/config.yml --output-dir exports/neus-facto-dtu65/ --num-points 1000 --remove-outliers True --use-bounding-box True --bounding-box-min -1 -1 -1 --bounding-box-max 1 1 1

In [ ]:
# look at mesh
!open3d draw exports/neus-facto-dtu65/mesh.ply

<table><tr>
<td> <img src="pics/dtu65-20000.jpg" height="300"/>  </td>
</tr></table>


#### Custom Training

In [ ]:
import os

# give experiment a name
data_name = 'mouse-2'

# define custom data folder location
data_input_dir = os.path.join('data', 'mine', data_name)
data_output_dir = data_input_dir + '-ss'

In [ ]:
# convert nerfstudio (COLMAP) to sdfstudio format, check images if conversion was succesfull
!python sdfstudio/scripts/datasets/process_nerfstudio_to_sdfstudio.py --data $data_input_dir --output-dir $data_output_dir --data-type colmap --scene-type object

In [ ]:
# show list of available models
!ns-train --help

For different model options see here: [Models](https://vscode.dev/github/autonomousvision/sdfstudio/blob/master/nerfstudio/configs/method_configs.py#L107). Relevant options for me are:

- "neuralangelo": "Implementation of Neuralangelo"
- "bakedangelo": "Implementation of Neuralangelo with BakedSDF"
- "neus-facto-angelo": "Implementation of Neuralangelo with neus-facto"
- "instant-ngp": "Implementation of Instant-NGP. Recommended real-time model for bounded synthetic data."
- "mipnerf": "High quality model for bounded scenes. (slow)"
    

##### Train NeusFacto

In [ ]:
# select model
model_name = 'neus-facto'

# experiment id
experiment_id = '2023-07-28_181851'

In [ ]:
output_dir = os.path.join('outputs', data_name, model_name, experiment_id)

In [14]:
# train neus-facto model from scratch on the dtu dataset scan65, default iteration is 20000
!ns-train neus-facto \
    --trainer.max-num-iterations 20000 \
    --viewer.websocket-port 7009 \
    --pipeline.model.sdf-field.inside-outside False \
    --pipeline.model.sdf-field.bias 0.3  \
    --pipeline.model.background-model mlp \
    --vis viewer \
    --experiment-name $data_name \
    nerfstudio-data --data $data_input_dir

Step (% Done)       Train Iter (time)    Train Rays / Sec     Vis Rays / Sec                         
-----------------------------------------------------------------------------------                  
8270 (41.35%)       128.581 ms           15.93 K                                                     
8280 (41.40%)       128.446 ms           15.95 K                                                     
8290 (41.45%)       127.506 ms           16.06 K                                                     
8300 (41.50%)       127.421 ms           16.07 K                                                     
8310 (41.55%)       128.447 ms           15.95 K                                                     
8320 (41.60%)       129.008 ms           15.88 K                                                     
8330 (41.65%)       128.689 ms           15.92 K              20.09 K                                
8340 (41.70%)       126.828 ms           16.16 K                                  

In [ ]:
# extract mesh
!ns-extract-mesh --load-config $output_dir/config.yml --output-path exports/neus-facto-dtu65/mesh.ply

In [ ]:
# look at mesh
!open3d draw exports/neus-facto-dtu65/mesh.ply

##### Train MipNeRF

In [ ]:
# select model
model_name = 'mipnerf'

# experiment id
experiment_id = '2023-07-28_112155'

In [ ]:
output_dir = os.path.join('outputs', data_name, model_name, experiment_id)

In [ ]:
# train model from scratch on the dtu dataset scan65, default iteration is 20000

# lead from checkpoint: --trainer.load-dir $output_dir/sdfstudio_models \

!ns-train mipnerf \
    --trainer.load-dir $output_dir/sdfstudio_models \
    --vis viewer \
    --trainer.max-num-iterations 20000 \
    --viewer.websocket-port 7008 \
    --experiment-name $data_name\
    nerfstudio-data --data $data_input_dir

In [ ]:
output_dir

In [ ]:
# extract mesh
!ns-extract-mesh --load-config $output_dir/config.yml --output-path exports/$data_name/mesh.ply

In [ ]:
# look at mesh
!open3d draw exports/$data_name/mesh.ply

#### Others

In [ ]:
# train MonoSDF
!ns-train monosdf \
    --pipeline.model.sdf-field.use-grid-feature True 
    --pipeline.model.sdf-field.hidden-dim 256 \
    --pipeline.model.sdf-field.num-layers 2 
    --pipeline.model.sdf-field.num-layers-color 2 \
    --pipeline.model.sdf-field.use-appearance-embedding True 
    --pipeline.model.sdf-field.geometric-init True \
    --pipeline.model.sdf-field.inside-outside True  \
    --pipeline.model.sdf-field.bias 0.8 \
    --pipeline.model.sdf-field.beta-init 0.1 \
    --pipeline.datamanager.train-num-images-to-sample-from 1 \
    --pipeline.datamanager.train-num-times-to-repeat-images 0 \
    --trainer.steps-per-eval-image 5000 \
    --pipeline.model.background-model none \
    --vis viewer \
    --experiment-name monosdf-htnt-scan1 \
    --pipeline.model.mono-depth-loss-mult 0.001 \
    --pipeline.model.mono-normal-loss-mult 0.01 \
    --pipeline.datamanager.train-num-rays-per-batch 2048 \
    --machine.num-gpus 8 sdfstudio-data \
    --data data/tanks-and-temple-highres/scan1 
    --include_mono_prior True --skip_every_for_val_split 30 \
`